In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install line-bot-sdk


In [6]:
from flask import Flask, request, abort
from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,    
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)
import pandas as pd
import os

# 從環境變數中獲取 Token 和 Secret
line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')

# 初始化 Flask
app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)
# 台灣所有縣市的代碼
cities = {
    "台北": "63",
    "新北": "65",
    "基隆": "10017",
    "桃園": "68",
    "新竹市": "10018",
    "新竹縣": "10004",
    "苗栗": "10005",
    "台中": "66",
    "彰化": "10007",
    "南投": "10008",
    "雲林": "10009",
    "嘉義市": "10020",
    "嘉義縣": "10010",
    "台南": "67",
    "高雄": "64",
    "屏東": "10013",
    "宜蘭": "10002",
    "花蓮": "10015",
    "台東": "10014",
    "澎湖": "10016",
    "金門": "09020",
    "連江": "09007"
}

# 讀取旅館資料
hotel_data = pd.read_csv('./Hotel_E_F.csv')
food_data = pd.read_csv('./Taiwan_Food_5000.csv')
attraction_data = pd.read_csv('./Taiwan_Attractions_5000.csv')
# 使用者選擇的城市和地區紀錄
user_city = {}

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    except Exception as e:
        print(f"Error: {e}")
        abort(400)
        
    return 'OK'

# 處理文字訊息
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_message = event.message.text.strip()
    user_id = event.source.user_id

    # 每次事件都初始化 API Client
    with ApiClient(Configuration(access_token=line_access_token)) as api_client:
        line_bot_api = MessagingApi(api_client)
        
        if user_message == "選擇城市":
            reply_text = (
                "請選擇想去的縣市：\n台北、新北、基隆、桃園、新竹市、新竹縣、苗栗、台中、彰化、南投、雲林、嘉義市、嘉義縣、台南、高雄、屏東、宜蘭、花蓮、台東、澎湖、金門、連江\n"
                "格式：<城市> <地區>\n例如：台北 士林區 或 桃園 中壢區"
            )
        elif user_message == "指令說明":
            reply_text = (
                "【指令清單】\n"
                "1. 選擇城市：設定查詢的城市與地區\n"
                "2. 景點推薦：查詢該地區的熱門景點\n"
                "3. 美食推薦：查詢該地區的美食推薦\n"
                "4. 住宿資訊：查詢該地區的旅館資訊\n"
                "5. 即時資訊：顯示當前天氣\n"
                "6. 指令說明：顯示所有可用指令"
            )
        elif " " in user_message:
            city, district = user_message.split(" ", 1)
            if city in cities:
                user_city[user_id] = {"city": city, "district": district}
                reply_text = f"您已選擇城市：{city}，地區：{district}\n接下來可以輸入「景點推薦」、「美食推薦」、「住宿資訊」或「即時資訊」來查詢資料。"
            else:
                reply_text = "無效的城市名稱，請重新輸入。"
        elif user_message == "即時資訊":
            if user_id in user_city:
                city_info = user_city[user_id]
                city_code = cities[city_info['city']]
                weather_url = f"https://www.cwa.gov.tw/V8/C/W/County/County.html?CID={city_code}"
                reply_text = (
                    f"{city_info['city']} {city_info['district']} 的天氣：\n{weather_url}"
                )
            else:
                reply_text = "請先選擇一個城市和地區，再輸入「即時資訊」。"
        elif user_message == "住宿資訊":
            if user_id in user_city:
                city_info = user_city[user_id]
                city_hotels = hotel_data[(hotel_data['城市'] == city_info['district'])]
                if not city_hotels.empty:
                    reply_text = "【住宿資訊】\n"
                    for index, row in city_hotels.iterrows():
                        reply_text += f"🏨 {row['旅館名稱']}\n📞 {row['電話']}\n📍 {row['地址']}\n💰 價格：{row['最低房價']} ~ {row['最高房價']}\n---\n"
                else:
                    reply_text = f"找不到 {city_info['city']} {city_info['district']} 的住宿資訊。"
            else:
                reply_text = "請先選擇一個城市和地區，再輸入「住宿資訊」。"
        elif user_message == "美食推薦":
            if user_id in user_city:
                city_info = user_city[user_id]
                city_foods = food_data[(food_data['Town'] == city_info['district'])]
                if not city_foods.empty:
                    reply_text = "【美食推薦】\n"
                    for index, row in city_foods.iterrows():
                        reply_text += f"🥢 {row['Name_c']} 種類:{row['Type']}\n📞 {row['Phone']}\n📍 {row['Address']}\n💰 價格：NT${row['Price_Min']}-{row['Price_Max']}\n---\n"
                else:
                    reply_text = f"找不到 {city_info['city']} {city_info['district']} 的美食推薦。"
            else:
                reply_text = "請先選擇一個城市和地區，再輸入「美食推薦」。"
        elif user_message == "景點推薦":
            if user_id in user_city:
                city_info = user_city[user_id]
                city_attractions = attraction_data[(attraction_data['Town'] == city_info['district'])]
                if not city_attractions.empty:
                    reply_text = "【景點推薦】\n"
                    for index, row in city_attractions.iterrows():
                        reply_text += f"🏞️ {row['Name_c']}\n📞 {row['Phone']}\n📍 {row['Address']}\n---\n"
                else:
                    reply_text = f"找不到 {city_info['city']} {city_info['district']} 的景點推薦。"
            else:
                reply_text = "請先選擇一個城市和地區，再輸入「景點推薦」。"
        else:
            reply_text = "請輸入正確的指令或輸入「指令說明」查看支援的指令。"        
        
        # 回傳訊息
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply_text)]
            )
        )

if __name__ == "__main__":
    app.run(port=5000)
    


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Jun/2025 17:24:02] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2025 17:24:08] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2025 17:24:13] "POST / HTTP/1.1" 200 -
